In [1]:
import numpy as np
import pandas as pd
from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertModel, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_transformers import AdamW as BertAdam
from tqdm import tqdm, trange
import pickle as pkl
import math

Using TensorFlow backend.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cuda


In [3]:
model = BertModel.from_pretrained("bert-base-uncased")
model.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [4]:
f = open('../../data/train_mod/ap880212_t', 'r').readlines()

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [6]:
a = [tokenizer.tokenize(f[i]) for i in range(len(f))]
print (len(a))
print (len(a[0]))

171
491


In [7]:
# print (a)

In [8]:
b = [tokenizer.convert_tokens_to_ids(i) for i in a] 

Token indices sequence length is longer than the specified maximum sequence length for this model (1052 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (801 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (842 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (717 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (637 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (1230 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (927 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (778 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1076 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (726 > 512). Running this sequence through the model will result in indexing errors


In [9]:
# print (b)

In [10]:
input_ids = pad_sequences(b, maxlen=510, dtype="long", truncating="post", padding="post")

In [11]:
# for i in input_ids:
#     print (len(i) == 510)

In [12]:
# print (len(input_ids))
# print (len(input_ids[0]))
# print (input_ids[0])

In [13]:
input_ids = np.array(input_ids)
CLS_ID = np.ones([len(input_ids), 1]) * 101
SEP_ID = np.ones([len(input_ids), 1]) * 102
print (CLS_ID.shape)
print (SEP_ID.shape)
input_ids = np.concatenate((CLS_ID, input_ids), axis=1)
print (input_ids.shape)
input_ids = np.concatenate((input_ids, SEP_ID), axis=1)
print (input_ids.shape)
# print (input_ids[0])

(171, 1)
(171, 1)
(171, 511)
(171, 512)


In [14]:
attention_mask = input_ids.copy()
attention_mask[attention_mask > 0] = 1

In [15]:
segment_mask = np.ones(input_ids.shape)
print (segment_mask.shape)

(171, 512)


In [16]:
batch_size = 8
input_ids = input_ids[:batch_size]
attention_mask = attention_mask[:batch_size]
segment_mask = segment_mask[:batch_size]

In [17]:
print (input_ids.shape)

(8, 512)


In [18]:
# method_list = [func for func in dir(BertTokenizer) if callable(getattr(BertTokenizer, func))]
# print (method_list)

In [19]:
input_id_tensor = torch.tensor(input_ids, dtype=torch.long).to(device)
attention_tensor = torch.tensor(attention_mask, dtype=torch.long).to(device)
segment_tensor = torch.tensor(segment_mask, dtype=torch.long).to(device)

In [20]:
embed = model.forward(input_ids=input_id_tensor, attention_mask=attention_tensor, token_type_ids=segment_tensor)

In [21]:
print (type(embed))

<class 'tuple'>


In [23]:
embed[0].shape

torch.Size([8, 512, 768])

In [24]:
type(embed[1])

torch.Tensor

In [25]:
embed[1].shape

torch.Size([8, 768])

In [26]:
len(embed)

2